# Now that all necessary data have now been extracted, transformed, dumped to parquet files in a lake (AWS S3) and then loaded into a DWH (DuckDB/Motherduck) we can start doing our/ analyses and do further transformations and draw insights through querying

In [87]:
import duckdb
import os

from dotenv import load_dotenv
from pathlib import Path
from duckdb.typing import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Local connection

In [88]:
# # C:\Users\LARRY\Documents\Scripts\data-engineering-path\chronic-disease-analyses\chronic_disease_analyses_db.db
# conn = duckdb.connect("chronic_disease_analyses_db.db")

# remote connection

In [89]:
# Build paths inside the project like this: BASE_DIR / 'subdir'.
# use this only in development
print("loading env variables...")
env_dir = Path('./').resolve()
load_dotenv(os.path.join(env_dir, '.env'))
print("env variables loaded.\n")

loading env variables...
env variables loaded.



In [90]:
# jdbc:duckdb:md:chronic_disease_analyses_db
# duckdb:///md:chronic_disease_analyses_db
print("connecting to duckdb...")
conn = duckdb.connect(f"md:chronic_disease_analyses_db?motherduck_token={os.environ['MOTHERDUCK_TOKEN']}")
print("connected to duckdb.\n")

connecting to duckdb...
connected to duckdb.



In [91]:
tables = [
    table
    for table_tuple in conn.sql("""SHOW TABLES""").fetchall()
    for table in table_tuple 
]
tables

['CDI',
 'CDILocation',
 'CDIStratification',
 'DataValueType',
 'Population',
 'PopulationState',
 'PopulationStratification',
 'Question']

In [92]:
for table in tables:
    counts = conn.sql(f"""SELECT COUNT(*) FROM {table}""").fetchall()[0][0]
    print(counts)

678471
51
11
15
2526336
51
24
192


In [93]:
conn.sql("""
    SELECT * FROM CDILocation
""")

┌────────────┬────────────────┬─────────────────────┬────────────────────┐
│ LocationID │  LocationDesc  │      Latitude       │     Longitude      │
│  varchar   │    varchar     │       double        │       double       │
├────────────┼────────────────┼─────────────────────┼────────────────────┤
│ NM         │ New Mexico     │ -106.24058098499967 │  34.52088095200048 │
│ OK         │ Oklahoma       │  -97.52107021399968 │  35.47203135600046 │
│ GA         │ Georgia        │  -83.62758034599966 │  32.83968109300048 │
│ KY         │ Kentucky       │  -84.77497104799966 │ 37.645970271000465 │
│ ND         │ North Dakota   │ -100.11842104899966 │  47.47531977900047 │
│ WY         │ Wyoming        │ -108.10983035299967 │  43.23554134300048 │
│ TN         │ Tennessee      │  -85.77449091399967 │  35.68094058000048 │
│ LA         │ Louisiana      │  -92.44568007099969 │  31.31266064400046 │
│ NE         │ Nebraska       │  -99.36572062299967 │   41.6410409880005 │
│ AK         │ Alaska    

In [94]:
conn.sql("""
    SELECT * FROM CDIStratification
""")

┌─────────┬─────────────┬──────────────┬──────────────────┐
│   Sex   │  Ethnicity  │    Origin    │ StratificationID │
│ varchar │   varchar   │   varchar    │     varchar      │
├─────────┼─────────────┼──────────────┼──────────────────┤
│ Male    │ All         │ Both         │ B_M_ALL          │
│ Both    │ White       │ Not Hispanic │ NH_B_WHITE       │
│ Both    │ Black       │ Not Hispanic │ NH_B_BLACK       │
│ Both    │ AIAN        │ Not Hispanic │ NH_B_AIAN        │
│ Both    │ Multiracial │ Not Hispanic │ NH_B_MULTI       │
│ Both    │ NHPI        │ Not Hispanic │ NH_B_NHPI        │
│ Both    │ Other       │ Not Hispanic │ NH_B_OTHER       │
│ Female  │ All         │ Both         │ B_F_ALL          │
│ Both    │ All         │ Hispanic     │ H_B_ALL          │
│ Both    │ All         │ Both         │ B_B_ALL          │
│ Both    │ Asian       │ Not Hispanic │ NH_B_ASIAN       │
├─────────┴─────────────┴──────────────┴──────────────────┤
│ 11 rows                               

In [95]:
conn.sql("""
    SELECT * FROM DataValueType
""")

┌───────────────────┬──────────────────────────────────────────┐
│  DataValueTypeID  │              DataValueType               │
│      varchar      │                 varchar                  │
├───────────────────┼──────────────────────────────────────────┤
│ MEDIAN            │ Median                                   │
│ NMBR              │ Number                                   │
│ AGEADJRATE        │ Age-adjusted Rate                        │
│ PERCAPALC         │ Per capita alcohol consumption           │
│ USD               │ US Dollars                               │
│ AGEADJMEAN        │ Age-adjusted Mean                        │
│ PREV              │ Prevalence                               │
│ MEAN              │ Mean                                     │
│ AVGANNCRDRATE     │ Average Annual Crude Rate                │
│ CRDRATE           │ Crude Rate                               │
│ AGEADJPREV        │ Age-adjusted Prevalence                  │
│ AVGANNAGEADJRATE  │ Ave

In [96]:
conn.sql("""
    SELECT * FROM Question
""")

┌────────────┬─────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────┬──────────┬────────┐
│ QuestionID │ TopicID │                                                   Question                                                   │                      Topic                      │ AgeStart │ AgeEnd │
│  varchar   │ varchar │                                                   varchar                                                    │                     varchar                     │  double  │ double │
├────────────┼─────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────────────────────────────────────────────┼──────────┼────────┤
│ ART1_1     │ ART     │ Arthritis among adults aged >= 18 years                                                                      │ Arthritis                               

In [97]:
conn.sql("""
    SELECT * FROM Population
""")

┌─────────┬───────┬──────────────────┬───────┬────────────┐
│ StateID │  Age  │ StratificationID │ Year  │ Population │
│ varchar │ float │     varchar      │ int32 │   int64    │
├─────────┼───────┼──────────────────┼───────┼────────────┤
│ AL      │   0.0 │ NH_M_WHITE       │  2000 │      19270 │
│ AL      │   0.0 │ NH_M_WHITE       │  2001 │      19612 │
│ AL      │   0.0 │ NH_M_WHITE       │  2002 │      18731 │
│ AL      │   0.0 │ NH_M_WHITE       │  2003 │      18623 │
│ AL      │   0.0 │ NH_M_WHITE       │  2004 │      18659 │
│ AL      │   0.0 │ NH_M_WHITE       │  2005 │      18816 │
│ AL      │   0.0 │ NH_M_WHITE       │  2006 │      18877 │
│ AL      │   0.0 │ NH_M_WHITE       │  2007 │      19027 │
│ AL      │   0.0 │ NH_M_WHITE       │  2008 │      18937 │
│ AL      │   0.0 │ NH_M_WHITE       │  2009 │      18039 │
│ ·       │    ·  │     ·            │    ·  │          · │
│ ·       │    ·  │     ·            │    ·  │          · │
│ ·       │    ·  │     ·            │  

In [98]:
conn.sql("""
    SELECT * FROM PopulationState
""")

┌───────────────┬─────────┐
│     State     │ StateID │
│    varchar    │ varchar │
├───────────────┼─────────┤
│ Mississippi   │ MS      │
│ South Dakota  │ SD      │
│ Utah          │ UT      │
│ Kentucky      │ KY      │
│ California    │ CA      │
│ Nebraska      │ NE      │
│ New Hampshire │ NH      │
│ Delaware      │ DE      │
│ Minnesota     │ MN      │
│ Nevada        │ NV      │
│   ·           │ ·       │
│   ·           │ ·       │
│   ·           │ ·       │
│ Michigan      │ MI      │
│ Illinois      │ IL      │
│ Maryland      │ MD      │
│ Alaska        │ AK      │
│ Connecticut   │ CT      │
│ Vermont       │ VT      │
│ Massachusetts │ MA      │
│ Oklahoma      │ OK      │
│ West Virginia │ WV      │
│ Virginia      │ VA      │
├───────────────┴─────────┤
│   51 rows (20 shown)    │
└─────────────────────────┘

In [99]:
conn.sql("""
    SELECT * FROM PopulationStratification
""")

┌─────────┬─────────────┬──────────────┬──────────────────┐
│   Sex   │  Ethnicity  │    Origin    │ StratificationID │
│ varchar │   varchar   │   varchar    │     varchar      │
├─────────┼─────────────┼──────────────┼──────────────────┤
│ Female  │ Black       │ Not Hispanic │ NH_F_BLACK       │
│ Female  │ Asian       │ Not Hispanic │ NH_F_ASIAN       │
│ Female  │ NHPI        │ Hispanic     │ H_F_NHPI         │
│ Male    │ Asian       │ Not Hispanic │ NH_M_ASIAN       │
│ Male    │ White       │ Not Hispanic │ NH_M_WHITE       │
│ Male    │ AIAN        │ Not Hispanic │ NH_M_AIAN        │
│ Male    │ Multiracial │ Hispanic     │ H_M_MULTI        │
│ Male    │ Black       │ Not Hispanic │ NH_M_BLACK       │
│ Male    │ Black       │ Hispanic     │ H_M_BLACK        │
│ Female  │ Asian       │ Hispanic     │ H_F_ASIAN        │
│  ·      │   ·         │    ·         │     ·            │
│  ·      │   ·         │    ·         │     ·            │
│  ·      │   ·         │    ·         │

In [100]:
conn.sql("""
    SELECT COUNT(*) 
    FROM PopulationState
    -- WHERE COLUMNS(*) IS NOT NULL
""")


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           51 │
└──────────────┘

In [101]:
conn.sql("""
    SELECT *
    FROM CDI
    ORDER BY LogID DESC
    LIMIT 10
""")

┌───────────┬─────────┬────────────┬───────────────┬───────────┬────────────────────┬─────────────────────┬────────────┬─────────────────┬──────────────────┬────────┐
│ YearStart │ YearEnd │ LocationID │ DataValueUnit │ DataValue │ LowConfidenceLimit │ HighConfidenceLimit │ QuestionID │ DataValueTypeID │ StratificationID │ LogID  │
│   int32   │  int32  │  varchar   │    varchar    │  double   │       double       │       double        │  varchar   │     varchar     │     varchar      │ int32  │
├───────────┼─────────┼────────────┼───────────────┼───────────┼────────────────────┼─────────────────────┼────────────┼─────────────────┼──────────────────┼────────┤
│      2012 │    2012 │ MS         │ %             │      75.3 │               71.2 │                79.0 │ TOB9_0     │ PRCT            │ B_B_ALL          │ 678471 │
│      2018 │    2018 │ MS         │ %             │      72.5 │               67.8 │                76.7 │ TOB9_0     │ PRCT            │ B_B_ALL          │ 678470 

In [102]:
conn.sql("""
    -- Create a 2nd CTE that will join the necessary
    -- values from the dimension tables to the fact
    -- table
    WITH MergedCDI AS (
    SELECT
        c.LogID AS ID,
        c.YearStart AS YearStart, 
        c.YearEnd AS YearEnd, 
        cl.LocationDesc AS LocationDesc, 
        c.DataValueUnit AS DataValueUnit,
        c.DataValue AS DataValue,
        q.Question AS Question,
        q.AgeStart AS AgeStart,
        q.AgeEnd AS AgeEnd,
        dvt.DataValueType AS DataValueType,
        cs.Sex AS Sex,
        cs.Ethnicity AS Ethnicity,
        cs.Origin AS Origin
    FROM CDI c
    LEFT JOIN CDILocation cl
    ON c.LocationID = cl.LocationID
    LEFT JOIN Question q
    ON c.QuestionID = q.QuestionID
    LEFT JOIN DataValueType dvt
    ON c.DataValueTypeID = dvt.DataValueTypeID
    LEFT JOIN CDIStratification cs
    ON c.StratificationID = cs.StratificationID
    WHERE q.AgeStart IS NOT NULL AND q.AgeEnd IS NOT NULL
    )

    -- groups the rows with the same YearStart, YearEnd, 
    -- LocationDesc, AgeStart, AgeEnd, Sex, Ethnicity, and Origin
    -- as this will simulate the rows taking on different values
    -- which we will need to dynamically aggregate the Population
    -- table
    SELECT 
    MAX(YearStart) AS YearStart, 
    MAX(YearEnd) AS YearEnd, 
    MAX(LocationDesc) AS LocationDesc, 
    MAX(AgeStart) AS AgeStart, 
    MAX(AgeEnd) AS AgeEnd, 
    MAX(Sex) AS Sex, 
    MAX(Ethnicity) AS Ethnicity, 
    MAX(Origin) AS Origin
    FROM MergedCDI
    GROUP BY (
    YearStart, 
    YearEnd, 
    LocationDesc, 
    AgeStart, 
    AgeEnd,
    Sex,
    Ethnicity,
    Origin
    )
""")

┌───────────┬─────────┬────────────────┬──────────┬────────┬─────────┬───────────┬──────────────┐
│ YearStart │ YearEnd │  LocationDesc  │ AgeStart │ AgeEnd │   Sex   │ Ethnicity │    Origin    │
│   int32   │  int32  │    varchar     │  double  │ double │ varchar │  varchar  │   varchar    │
├───────────┼─────────┼────────────────┼──────────┼────────┼─────────┼───────────┼──────────────┤
│      2019 │    2019 │ Connecticut    │     18.0 │    inf │ Both    │ All       │ Hispanic     │
│      2020 │    2020 │ Vermont        │     18.0 │    inf │ Male    │ All       │ Both         │
│      2015 │    2015 │ Alaska         │     18.0 │    inf │ Female  │ All       │ Both         │
│      2015 │    2015 │ North Carolina │     18.0 │    inf │ Both    │ Black     │ Not Hispanic │
│      2020 │    2020 │ Delaware       │     18.0 │    inf │ Both    │ White     │ Not Hispanic │
│      2016 │    2016 │ Ohio           │     18.0 │    inf │ Male    │ All       │ Both         │
│      2017 │    201

In [103]:
conn.sql("""
    WITH MergedPopulation AS (
    SELECT 
        ps.State AS State,
        p.Age AS Age,
        p.Year AS Year,
        pstr.Sex AS Sex,
        pstr.Ethnicity AS Ethnicity,
        pstr.Origin AS Origin,
        p.Population AS Population
    FROM Population p
    LEFT JOIN PopulationState ps
    ON p.StateID = ps.StateID
    LEFT JOIN PopulationStratification pstr
    ON p.StratificationID = pstr.StratificationID
    )

    SELECT Year, SUM(Population) AS TotalPopulation
    FROM MergedPopulation
    WHERE (Year BETWEEN 2011 AND 2013) AND
    (Age BETWEEN 18 AND 85) AND
    State = 'Wyoming' AND
    Sex IN ('Male') AND
    Ethnicity IN ('White', 'Black', 'AIAN', 'Asian', 'NHPI', 'Multiracial') AND
    Origin IN ('Hispanic',)
    GROUP BY Year
    ORDER BY Year ASC
""")

┌───────┬─────────────────┐
│ Year  │ TotalPopulation │
│ int32 │     int128      │
├───────┼─────────────────┤
│  2011 │           17565 │
│  2012 │           18734 │
│  2013 │           19147 │
└───────┴─────────────────┘

I have loaded two fact tables CDI and Population and a couple dimension tables in DuckDB. I did joins on the CDI fact table and its respective dimension tables which yields a snippet of the table below CDI table merged

And below is the Population fact table merged with its other dimension tables yielding this snippet below Population table merged enter image description here

Now what I want to basically do is filter out the Population table based only on the values of this particular row of the CDI table. In this case the current row outlined in green will somehow do this query
```
SELECT Year, SUM(Population) AS TotalPopulation
FROM Population
WHERE (Year BETWEEN 2018 AND 2018) AND
(Age BETWEEN 18 AND 85) AND
State = 'Pennsylvania' AND
Sex IN ('Male', 'Female') AND
Ethnicity IN ('Multiracial') AND
Origin IN ('Not Hispanic')
GROUP BY Year
ORDER BY Year ASC
```

This query aggregates the Population column values based on the row values of the CDI table. What I'm just at a loss in trying to implement is doing this aggregation operation for all row values in the CDI table. Here is a full visualization of what I'm trying to do. enter image description here

How would I implement this type of varying filtering aggregation based on each row column values of the CDI table? I'm using DuckDB as the OLAP DB here so ANSI SQL is what I'm trying to use to implement this task. Could it be possible only using this kind of SQL?

In [104]:
conn.sql("""
    SELECT current_schema();
""")

┌──────────────────┐
│ current_schema() │
│     varchar      │
├──────────────────┤
│ main             │
└──────────────────┘

In [105]:
conn.sql("""
    SELECT * FROM duckdb_columns();
""")

┌─────────────────────────────┬──────────────┬─────────────┬────────────┬────────────────────┬───────────┬─────────────────────┬──────────────┬─────────┬──────────┬────────────────┬─────────────┬───────────┬──────────────┬──────────────────────────┬───────────────────┬─────────────────────────┬───────────────┐
│        database_name        │ database_oid │ schema_name │ schema_oid │     table_name     │ table_oid │     column_name     │ column_index │ comment │ internal │ column_default │ is_nullable │ data_type │ data_type_id │ character_maximum_length │ numeric_precision │ numeric_precision_radix │ numeric_scale │
│           varchar           │    int64     │   varchar   │   int64    │      varchar       │   int64   │       varchar       │    int32     │ varchar │ boolean  │    varchar     │   boolean   │  varchar  │    int64     │          int32           │       int32       │          int32          │     int32     │
├─────────────────────────────┼──────────────┼─────────────┼────

In [ ]:
def test_prototype(state):
    # query = f"""
    #     WITH MergedPopulation AS (
    #         SELECT 
    #             ps.State AS State,
    #             p.Age AS Age,
    #             p.Year AS Year,
    #             pstr.Sex AS Sex,
    #             pstr.Ethnicity AS Ethnicity,
    #             pstr.Origin AS Origin,
    #             p.Population AS Population
    #         FROM Population p
    #         LEFT JOIN PopulationState ps
    #         ON p.StateID = ps.StateID
    #         LEFT JOIN PopulationStratification pstr
    #         ON p.StratificationID = pstr.StratificationID
    #     )               
        
    #     SELECT SUM(Population) AS TotalPopulation
    #     FROM MergedPopulation
    #     WHERE State = '{state}'
    # """
    # total_population = conn.sql(query).fetchall()[-1][-1]
    total_population = len(state)
    return total_population

In [107]:
conn.create_function("test_prototype", test_prototype, [VARCHAR], BIGINT)

CatalogException: Catalog Error: Scalar Function with name "test_prototype" already exists!

In [108]:
conn.remove_function("test_prototype")

InvalidInputException: Invalid Input Error: No function by the name of 'test_prototype' was found in the list of registered functions

In [111]:
conn.sql("""
    -- Create a 2nd CTE that will join the necessary
    -- values from the dimension tables to the fact
    -- table
    WITH MergedCDI AS (
    SELECT
        c.LogID AS ID,
        c.YearStart AS YearStart, 
        c.YearEnd AS YearEnd, 
        cl.LocationDesc AS LocationDesc, 
        c.DataValueUnit AS DataValueUnit,
        c.DataValue AS DataValue,
        q.Question AS Question,
        q.AgeStart AS AgeStart,
        q.AgeEnd AS AgeEnd,
        dvt.DataValueType AS DataValueType,
        cs.Sex AS Sex,
        cs.Ethnicity AS Ethnicity,
        cs.Origin AS Origin
    FROM CDI c
    LEFT JOIN CDILocation cl
    ON c.LocationID = cl.LocationID
    LEFT JOIN Question q
    ON c.QuestionID = q.QuestionID
    LEFT JOIN DataValueType dvt
    ON c.DataValueTypeID = dvt.DataValueTypeID
    LEFT JOIN CDIStratification cs
    ON c.StratificationID = cs.StratificationID
    WHERE q.AgeStart IS NOT NULL AND q.AgeEnd IS NOT NULL
    ),

    -- groups the rows with the same YearStart, YearEnd, 
    -- LocationDesc, AgeStart, AgeEnd, Sex, Ethnicity, and Origin
    -- as this will simulate the rows taking on different values
    -- which we will need to dynamically aggregate the Population
    -- table
    FilteredMergedCDI AS (
        SELECT 
            MAX(YearStart) AS YearStart, 
            MAX(YearEnd) AS YearEnd, 
            MAX(LocationDesc) AS LocationDesc, 
            MAX(AgeStart) AS AgeStart, 
            MAX(AgeEnd) AS AgeEnd, 
            MAX(Sex) AS Sex, 
            MAX(Ethnicity) AS Ethnicity, 
            MAX(Origin) AS Origin
        FROM MergedCDI
        GROUP BY (
            YearStart, 
            YearEnd, 
            LocationDesc, 
            AgeStart, 
            AgeEnd,
            Sex,
            Ethnicity,
            Origin
        )
    )
         
    SELECT test_prototype(LocationDesc) AS TotalPopulation
    FROM FilteredMergedCDI
""")

ConnectionException: Connection Error: Connection already closed!

In [110]:
conn.close()